In [1]:
import tensorflow as tf
from tensorflow import keras
import ipywidgets

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version

Num GPUs Available:  1
2.12.0


'3.9.16 (main, Dec  7 2022, 01:11:51) \n[GCC 9.4.0]'

In [ ]:
# ! pip install opencv-python

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [3]:
import os
import io
import matplotlib.pyplot as plt
import cv2
import imageio
#import medmnist
#import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
np.random.seed(SEED)

## Hyperparameters

The hyperparameters are chosen via hyperparameter
search. You can learn more about the process in the "conclusion" section.

In [4]:
#Binray

# DATA
DATASET_NAME = "LAD2000"
BATCH_SIZE = 32
PATH = '/content/gdrive/MyDrive/Datasets/LAD-2000-correct'
# AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (75, 75,15, 3)
NUM_CLASSES = 2

# OPTIMIZER
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:
#binary_data_path = '/content/gdrive/MyDrive/Datasets/UCF-original/Binary_Classification/Final_Binary_Data/'
#X_train = np.concatenate((np.load(binary_data_path + 'Train/Non_violence/non_X_train.npy'), np.load(binary_data_path + 'Train/Violence/V_X_train.npy')))

#Load data

In [5]:
X_train= np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Train/X.npy',allow_pickle=True, mmap_mode="r")
y_train = np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Train/y.npy', allow_pickle=True,mmap_mode = "r")

In [6]:
X_val = np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Val/X.npy',allow_pickle=True, mmap_mode = "r")
y_val = np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Val/y.npy', allow_pickle=True, mmap_mode = "r")

In [ ]:
X_test = np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Test/X.npy',allow_pickle=True, mmap_mode = "r")


In [16]:
y_test = np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Test/y.npy', allow_pickle=True, mmap_mode = "r")

#Optical-flow calculation


In [7]:
import cv2
from google.colab.patches import cv2_imshow
def opticalflow(X_data, y_data):
    
    optical_flow = []
    flow_labels = []
    for i in range (X_data.shape[0]):
        packet = []
        if i == 1487:
            print(i)
        for j in range(1, X_data.shape[1]):
            frame1 = X_data[i][j-1]
            frame2 = X_data[i][j]
            gray1 = cv2.cvtColor((255*frame1).astype(np.uint8), cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor((255*frame2).astype(np.uint8), cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            packet.append(flow)
        optical_flow.append(np.asarray(packet))
        flow_labels.append(y_data[i])

            # # display the optical flow field
            # h, w = gray1.shape
            # y, x = np.mgrid[0:h:5, 0:w:5].reshape(2, -1)
            # fx, fy = flow[y, x].T
            # lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
            # lines = np.int32(lines + 0.5)
            # vis = cv2.cvtColor(gray2, cv2.COLOR_GRAY2BGR)
            # cv2.polylines(vis, lines, 0, (0, 255, 0))
            # for (x1, y1), (_x2, _y2) in lines:
            #     cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
            # cv2_imshow(vis)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

    return np.asarray(optical_flow), np.asarray(flow_labels)

In [ ]:
X_train_opt.shape

(4476, 14, 75, 75, 2)

In [7]:
X_train = np.moveaxis(X_train, 1, 3)
X_val = np.moveaxis(X_val, 1, 3)
#X_test = np.moveaxis(X_test, 1, 3)

In [10]:
print(X_train.shape, X_val.shape)

(4476, 75, 75, 15, 3) (1002, 75, 75, 15, 3)


In [11]:
dim = X_train.shape
dim = dim[1:]
print(dim)

(75, 75, 15, 3)


In [13]:
y_train[7]

0.0

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_val = np.asarray(X_val).astype(np.float32)
#X_test = np.asarray(X_test).astype(np.float32)

In [ ]:
y_train = np.asarray(y_train).astype(np.float32)
y_val = np.asarray(y_val).astype(np.float32)


In [17]:
y_test = np.asarray(y_test).astype(np.float32)

#Binary with MM model


In [12]:
import os
import sys
import cv2
import zipfile
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
# from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
from tensorflow.keras import layers, models

In [13]:


model = models.Sequential()
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform', data_format='channels_last', input_shape=dim))
model.add(layers.MaxPooling3D((2, 2, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling3D((3, 3, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling3D((2, 2, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Flatten())
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))



In [14]:


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00025),
              metrics=['accuracy'])



### `tf.data` pipeline

In [7]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, x_in, y_in, batch_size=BATCH_SIZE, shuffle=True):
        # Initialization
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.x = x_in
        self.y = y_in
        self.datalen = len(y_in)
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        # get batch indexes from shuffled indexes
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        x_batch = self.x[batch_indexes]
        y_batch = self.y[batch_indexes]
        return x_batch, y_batch
    
    def __len__(self):
        # Denotes the number of batches per epoch
        return self.datalen // self.batch_size

    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(self.datalen)
        if self.shuffle:
            np.random.shuffle(self.indexes)

training_generator = DataGenerator(np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Train/X.npy',allow_pickle=True, mmap_mode="r"), np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Train/y.npy',allow_pickle=True, mmap_mode="r"))
validation_generator = DataGenerator(np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Val/X.npy',allow_pickle=True, mmap_mode="r"), np.load('/content/gdrive/MyDrive/All extracted frames/LAD-2000-correct/Val/y.npy',allow_pickle=True, mmap_mode="r"))

#Vivit


In [8]:

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [9]:

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens


In [ ]:
# model_checkpoint = ModelCheckpoint("../checkpoints/vivit-mini-ucf/weights.{epoch:02d}.hdf5",
#                 monitor='val_loss',
#                 save_weights_only=True,
#                 save_best_only=True)

In [10]:
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_lad/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

In [11]:
from keras.callbacks import ReduceLROnPlateau
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_lad/checkpoint/'
checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

earlystopping = tf.keras.callbacks.EarlyStopping(
                     monitor="val_loss",
                     min_delta=0.025,
                     patience=10,
                     verbose=0,
                     mode="min",
                     baseline=None,
                     restore_best_weights=False
                 )

values = np.linspace(0.00001,LEARNING_RATE,14)[::-1].astype(np.float32)
boundaries = np.linspace(5, 45,13)[:values.shape[0]-1].astype(np.int32)

scheduler = keras.optimizers.schedules.PiecewiseConstantDecay(
    list(boundaries), list(values))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)

callbacks = [checkpoint]

In [12]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 2)
y_val = to_categorical(y_val, 2)
#y_test_opt = to_categorical(y_test_opt,2)
y_train.shape

(4476, 2)

In [13]:
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [14]:
def run_experiment():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="binary_crossentropy",
            metrics=[
                keras.metrics.BinaryAccuracy(name="accuracy")
            ],
        )

        # early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, mode='min')

        # Train the model.
        history = model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_val, y_val), callbacks=[model_checkpoint, callbacks])

        # _, accuracy, top_5_accuracy = model.evaluate(testloader)
        # print(f"Test accuracy: {round(accuracy * 100, 2)}%")
        # print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model, history


model, history = run_experiment()

Epoch 1/100
140/140 [==============================] - 70s 208ms/step - loss: 0.7469 - accuracy: 0.4989 - val_loss: 0.7337 - val_accuracy: 0.5000
Epoch 2/100
140/140 [==============================] - 12s 89ms/step - loss: 0.7009 - accuracy: 0.4993 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 3/100
140/140 [==============================] - 12s 84ms/step - loss: 0.6961 - accuracy: 0.4973 - val_loss: 0.7005 - val_accuracy: 0.5000
Epoch 4/100
140/140 [==============================] - 12s 88ms/step - loss: 0.6953 - accuracy: 0.5047 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 5/100
140/140 [==============================] - 10s 73ms/step - loss: 0.6942 - accuracy: 0.4942 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 6/100
140/140 [==============================] - 11s 77ms/step - loss: 0.6942 - accuracy: 0.4908 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 7/100
140/140 [==============================] - 12s 87ms/step - loss: 0.6940 - accuracy: 0.4989 - val_loss: 0.6932 - val_a

#another cnn model


In [ ]:
import os
import sys
import cv2
import zipfile
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
# from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
from tensorflow.keras import layers, models

In [ ]:
def create_model(dim):
  model = models.Sequential()
  # model.add(layers.BatchNormalization())
  model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform', data_format='channels_last', input_shape=dim))
  model.add(layers.MaxPooling3D((2, 2, 1)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Conv3D(32, kernel_size=(3, 3, 2), activation='relu', kernel_initializer='he_uniform'))
  model.add(layers.MaxPooling3D((3, 3, 1)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform'))
  model.add(layers.MaxPooling3D((2, 2, 1)))
  # model.add(layers.BatchNormalization())
  model.add(layers.Flatten())
  # model.add(layers.Dropout(0.5))
  model.add(layers.Dense(4096, activation='relu'))
  model.add(layers.Dense(1024, activation='relu'))
  # model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  # model.add(layers.BatchNormalization())
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

In [ ]:
model2 = model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

In [ ]:
model = create_model(dim)
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_1 (Conv3D)           (None, 73, 73, 14, 32)    608       
                                                                 
 max_pooling3d (MaxPooling3D  (None, 36, 36, 14, 32)   0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 34, 34, 13, 32)    18464     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 11, 11, 13, 32)   0         
 3D)                                                             
                                                                 
 conv3d_3 (Conv3D)           (None, 9, 9, 13, 32)      9248      
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 4, 4, 13, 32)    

In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_flow3/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

In [ ]:
from keras.callbacks import ReduceLROnPlateau
checkpoint_filepath = '/content/gdrive/MyDrive/Checkpoints/run_flow3/checkpoint/'
model_checkpoint = model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                filepath=checkpoint_filepath,
                                save_weights_only=True,
                                monitor='val_loss',
                                mode='min',
                                save_best_only=True
                                )

earlystopping = tf.keras.callbacks.EarlyStopping(
                     monitor="val_loss",
                     min_delta=0.025,
                     patience=10,
                     verbose=0,
                     mode="min",
                     baseline=None,
                     restore_best_weights=False
                 )

values = np.linspace(0.00001,LEARNING_RATE,14)[::-1].astype(np.float32)
boundaries = np.linspace(5, 45,13)[:values.shape[0]-1].astype(np.int32)

scheduler = keras.optimizers.schedules.PiecewiseConstantDecay(
    list(boundaries), list(values))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)

callbacks = [checkpoint]

In [ ]:
import gc
gc.collect()

1626

In [ ]:
training_generator

## Train

In [ ]:

def run_experiment():
    mirrored_strategy = tf.distribute.MirroredStrategy(devices= ["/gpu:0"],cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    with mirrored_strategy.scope():
    # Initialize model
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        # Compile the model with the optimizer, loss function
        # and the metrics.
        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

        # early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, mode='min')

        # Train the model.
        history = model.fit(training_generator, epochs=EPOCHS, validation_data=validation_generator, callbacks=[model_checkpoint, callbacks])

        # _, accuracy, top_5_accuracy = model.evaluate(testloader)
        # print(f"Test accuracy: {round(accuracy * 100, 2)}%")
        # print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model, history


model, history = run_experiment()

Epoch 1/100
139/139 [==============================] - 68s 224ms/step - loss: 0.7918 - accuracy: 0.5153 - top-5-accuracy: 1.0000 - val_loss: 0.7033 - val_accuracy: 0.5161 - val_top-5-accuracy: 1.0000
Epoch 2/100
139/139 [==============================] - 14s 98ms/step - loss: 0.6934 - accuracy: 0.5382 - top-5-accuracy: 1.0000 - val_loss: 0.6985 - val_accuracy: 0.5111 - val_top-5-accuracy: 1.0000
Epoch 3/100
139/139 [==============================] - 13s 92ms/step - loss: 0.6897 - accuracy: 0.5486 - top-5-accuracy: 1.0000 - val_loss: 0.7014 - val_accuracy: 0.5171 - val_top-5-accuracy: 1.0000
Epoch 4/100
139/139 [==============================] - 12s 90ms/step - loss: 0.6907 - accuracy: 0.5364 - top-5-accuracy: 1.0000 - val_loss: 0.6966 - val_accuracy: 0.5222 - val_top-5-accuracy: 1.0000
Epoch 5/100
139/139 [==============================] - 12s 89ms/step - loss: 0.6878 - accuracy: 0.5445 - top-5-accuracy: 1.0000 - val_loss: 0.6889 - val_accuracy: 0.5665 - val_top-5-accuracy: 1.0000
Epoc